In [2]:
from models import build_model, Permute, Covxvit
import torch
import torch.nn as nn
from torchvision import models

num_classes = 10
# model = build_model(10, model="swin").cpu()
# premodel = build_model(10, model="convnext").cpu()


In [20]:
# vit = models.vit_l_16(weights = models.ViT_L_16_Weights.IMAGENET1K_V1)
# cnn = models.convnext_large(weights=models.ConvNeXt_Large_Weights.IMAGENET1K_V1)
# model = build_model(num_classes, model = "covxnext", meta = [10,10,10])
# model(torch.zeros((16,3,224,224)), torch.zeros((16,3), dtype=torch.long))
# model = build_model(10, model="conxvit")
# model(torch.zeros((2,3,224,224)))
# cnn = models.convnext_large()
cnn.features[:6]#(torch.zeros((1,3,224,224))).shape

Sequential(
  (0): Conv2dNormActivation(
    (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
  )
  (1): Sequential(
    (0): CNBlock(
      (block): Sequential(
        (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
        (1): Permute()
        (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
        (3): Linear(in_features=192, out_features=768, bias=True)
        (4): GELU(approximate='none')
        (5): Linear(in_features=768, out_features=192, bias=True)
        (6): Permute()
      )
      (stochastic_depth): StochasticDepth(p=0.0, mode=row)
    )
    (1): CNBlock(
      (block): Sequential(
        (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
        (1): Permute()
        (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
        (3): Linear(in_features=192, out_features=768, bias=True)
        (4): GELU(approx

In [ ]:
import numpy as np
X = np.array(["A"] * 10 + ["B"] * 20 + ["C"] * 30 + ["D"] * 40, dtype=object).reshape(-1, 1)
y = np.array([0] * 20 + [1] * 70 + [2] * 10, dtype=np.int32)
from collections import Counter
print(f"Original class counts: {Counter(y)}")
from imblearn.over_sampling import SMOTEN
sampler = SMOTEN(random_state=0, sampling_strategy= {0:40, 1:70, 2:20})
X_res, y_res = sampler.fit_resample(X, y)
print(f"Class counts after resampling {Counter(y_res)}")
# X_res[:,0], y_res

Original class counts: Counter({np.int32(1): 70, np.int32(0): 20, np.int32(2): 10})
Class counts after resampling Counter({np.int32(0): 70, np.int32(1): 70, np.int32(2): 70})


(array(['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B',
        'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
        'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
        'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C',
        'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'D',
        'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D',
        'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D',
        'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'B', 'B', 'A', 'A',
        'A', 'A', 'A', 'B', 'B', 'A', 'A', 'B', 'A', 'A', 'A', 'B', 'B',
        'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'B', 'B', 'A', 'B', 'A',
        'B', 'B', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'B',
        'B', 'A', 'A', 'A', 'A', 'A', 'A', 'D', 'D', 'D', 'D', 'D', 'D',
        'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D',
        'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D'

In [1]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
class ISIC2019Dataset(Dataset):
    def __init__(self, img_dir, csv_file, meta_file, transform=None, train = False, filter = True):
        # Image directory and CSV file
        self.img_dir = img_dir
        self.data = pd.read_csv(csv_file)
        meta = pd.read_csv(meta_file)
        self.data = self.data.merge(meta, how='inner', on='image')
        # print(self.data)
        self.transform = transform
        self.train = train

        # List of target classes
        self.label_cols = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"]

        if(filter):
            self.label_num = [len(self.data[self.data["label"] == label]) for label in self.label_cols]
            filtered = []
            self.count = {}
            for label, number in zip(self.label_cols, self.label_num):
                self.count[label] = number
                if(number < 1000):
                    filtered.append(label)
            self.minor = filtered
            self.label_num.sort()
            self.minimum = self.label_num[int(len(self.label_num)/2)]
            self.total = sum(self.label_num)
            for label in self.minor:
                self.count[label] = self.minimum
            # self.data = pd.concat([self.data[self.data["label"]==label]for label in filtered], ignore_index=True)

        # Remove rows with unknown labels
        if "UNK" in self.data.columns:
            self.data = self.data[self.data[self.label_cols].sum(axis=1) > 0].reset_index(drop=True)
dataset = ISIC2019Dataset(img_dir="data/Training",
                            csv_file="splits/train_split.csv",
                            meta_file="data/ISIC_2019_Training_Metadata.csv")
dataset.data[["label", "age_approx", "anatom_site_general", "sex"]]

,label,age_approx,anatom_site_general,sex
0,NV,30.0,NaN,female
1,BCC,50.0,head/neck,male
2,MEL,65.0,posterior torso,male
3,BKL,70.0,head/neck,female
4,MEL,60.0,head/neck,male
...,...,...,...,...
17726,BKL,70.0,head/neck,female
17727,BCC,75.0,head/neck,male
17728,NV,50.0,anterior torso,male
17729,BCC,30.0,anterior torso,female


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline

num_cols = ["age_approx"]
cat_cols = ["anatom_site_general", "sex"]
fill_na = {col: "missing" for col in cat_cols}
fill_na["age_approx"] =dataset.data["age_approx"].median()
dataset.data.fillna(fill_na, inplace=True)

In [53]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
cat_feature_indices = [1,2]

pipe = SMOTENC(
        categorical_features=cat_feature_indices,
        random_state=42,
        sampling_strategy=dataset.count
    )



min_age, max_age = np.min(dataset.data["age_approx"]), np.max(dataset.data["age_approx"])
def discretize(x):
    return np.clip(np.round(x.astype(np.float16) / 5) * 5, min_age, max_age)

X_res, y_res = pipe.fit_resample(dataset.data[["age_approx", "anatom_site_general", "sex"]], dataset.data[["label"]])
X_res = X_res.replace("missing", np.nan)
X_res.iloc[dataset.total:, 0] = discretize(X_res.iloc[dataset.total:, 0])
y_res = y_res[dataset.total:].values.ravel()
df_resampled = pd.DataFrame(
    X_res.iloc[dataset.total:, :],
    columns=["age_approx", "anatom_site_general", "sex"]
)
label_cols = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"]

encoder = OneHotEncoder(categories=[label_cols], sparse_output=False)

labels_encoded = encoder.fit_transform(pd.DataFrame(y_res, columns = ["label"]))

df_onehot = pd.DataFrame(labels_encoded, columns = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"])
df_resampled = df_resampled.reset_index(drop=True)
df_onehot = df_onehot.reset_index(drop=True)
df_merged = pd.concat([df_resampled, df_onehot], axis=1)
df_merged["label"] = y_res
df_merged = df_merged.reset_index(drop=True)
df_merged.to_csv("splits/meta_generated.csv", index=False)

In [38]:
from sklearn.preprocessing import OneHotEncoder

label_cols = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"]

encoder = OneHotEncoder(categories=[label_cols], sparse_output=False)

# 假設 df["label"] 是你的標籤欄位
labels_encoded = encoder.fit_transform(df_resampled[["label"]])

# 轉成 DataFrame
df_onehot = pd.DataFrame(labels_encoded, columns = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"])

print(df_onehot.head())
encoder.get_feature_names_out(["label"])


   MEL   NV  BCC   AK  BKL   DF  VASC  SCC
0  0.0  0.0  0.0  1.0  0.0  0.0   0.0  0.0
1  0.0  0.0  0.0  1.0  0.0  0.0   0.0  0.0
2  0.0  0.0  0.0  1.0  0.0  0.0   0.0  0.0
3  0.0  0.0  0.0  1.0  0.0  0.0   0.0  0.0
4  0.0  0.0  0.0  1.0  0.0  0.0   0.0  0.0


array(['label_MEL', 'label_NV', 'label_BCC', 'label_AK', 'label_BKL',
       'label_DF', 'label_VASC', 'label_SCC'], dtype=object)